In [ ]:
import tensorflow as tf
 
import IPython.display as display
 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
 
import numpy as np
import PIL.Image
import time
import functools

In [ ]:
content_layers = ['block5_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1', 
                'block5_conv1']


num_content_layers = len(content_layers)
num_style_layers = len(style_layers)


In [ ]:
def obter_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)


In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)


In [ ]:
class StyleTransfer(tf.keras.models.Model):

  def __init__(self, content_layers, style_layers) :
    super().__init__()

    self.content_layers = content_layers
    self.style_layers = style_layers
    self.num_content_layers = len(content_layers)
    self.num_style_layers = len(style_layers)
    self.model = obter_layers( content_layers + style_layers)
    self.model.trainable = False

  def __call__(self,inputs):

    inputs = inputs*255
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)

    outputs = self.model(preprocessed_input)

    content_output, style_outputs = (outputs[:num_content_layers],outputs[num_content_layers:])

    style_outputs = [gram_matrix(style_output) 
                      for style_output in style_outputs]
    
    content_dict = {content_name:valor
                    for content_name, valor  in zip(content_layers, content_output)}

    style_dict = {style_name:valor
                  for style_name, valor in zip(style_layers, style_outputs)}

    return {'content':content_dict , 'style':style_dict}
  
 

In [ ]:
content_image = load_img('content.jpeg')
style_image = load_img('style.jpg')

In [ ]:
transfer = StyleTransfer(content_layers, style_layers)

In [ ]:
content_features = transfer(content_image)['content']
style_features = transfer(style_image)['style']

In [ ]:
np.array(content_image).shape

(1, 384, 512, 3)

In [ ]:
imagem = tf.Variable(content_image) 

In [ ]:
#imagem = tf.Variable(np.float32(np.random.rand(1,384,512,3)))

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.01, beta_1=0.99, epsilon=1e-1) #learning 0.03

In [ ]:
 def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_features[name])**2) 
                          for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_features[name])**2) 
                            for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss


In [ ]:
@tf.function()
def train_step(imagem):
  with tf.GradientTape() as tape:
    outputs = transfer(imagem)
    loss = style_content_loss(outputs)
    loss += total_variation_weight*tf.image.total_variation(imagem)

  grad = tape.gradient(loss, imagem)
  opt.apply_gradients([(grad, imagem)])
  imagem.assign(clip_0_1(imagem))
  return loss

In [ ]:
style_weight=1e-2#0.7
content_weight=1e4 #100
total_variation_weight=30

In [ ]:
import time
start = time.time()
melhor_loss = 20000000000000000000000000
epochs = 50
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    loss = train_step(imagem)
    print(".", end='')
    if loss < melhor_loss:
      melhor_loss = loss
      img_final = imagem
      
  display.clear_output(wait=True)
  display.display(tensor_to_image(imagem))
  print("Train step: {}".format(step))
  
end = time.time()
print("Total time: {:.1f}".format(end-start))
file_name = 'out.png'
tensor_to_image(img_final).save(file_name)
tensor_to_image(img_final)


In [ ]:
file_name = 'out.png'
tensor_to_image(img_final).save(file_name)
tensor_to_image(img_final)